# Tutorial Keras - a Casa Feliz

Bem-vindo à primeira tarefa desta semana, nesta tarefa você irá:
1. Aprender a utilizar o Keras, um framework de programação de alto-nível para redes neurais, escrito em Python e capaz de rodar sobre vários outros frameworks de baixo nível como TensorFlow e CNTK. 
2. Ver como é possível, em poucas horas, construir um algoritmo de aprendizado profundo.

Porque utilizar o Keras? Keras foi desenvolvido para permitir que engenheiros de aprendizado profundo pudessem construir um experimento utilizando modelos diferentes de forma mais rápida. Assim como TensorFlow é um framework de nível mais alto do que o Python, Keras funciona em um nível ainda mais alto e prove abstrações adicionais, permitindo ir de idéias a resultados com o menor tempo possível, o que pode ser essencial na busca por um bom modelo. Embora Keras seja mais restritivo que os frameworks de baixo-nível na implementação de modelos complexos, ele funciona muito bem em modelos mais comuns. 

Neste exercício você irá trabalhar com o probelma da "Casa Feliz", que será explicado mais abaixo. Vamos carregar os pacotes necessários, lembrando que o Keras é um pacote que já vem incluso no TensorFlow, então, se a sua versão do TensroFlow for nova, você não tem que se preocupar em instalar o Keras, porém, se for mais antiga, você pode: atualizar a sua versão do TensorFlow ou instalar o pacote do Keras individualmente. 

In [1]:
import numpy as np
import tensorflow as tf
import pydot
from IPython.display import SVG
from kt_utils import *
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
%matplotlib inline

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


ModuleNotFoundError: No module named 'pydot'

Vamos importar agora os pacotes do Keras necessários para esta atividade

In [ ]:
# Importando os pacotes necessários do Keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from tensorflow.keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import plot_model
import tensorflow.keras.backend as K
tf.keras.backend.set_image_data_format('channels_last')

**Nota**: Como você pode ver nós importamos diversas funções do Keras. Você pode utilizar qualquer uma delas  simplesmente chamando a função desejada, por exemplo: `X = Input(...)` ou `X = ZeroPadding2D(...)`.

## 1 - A Casa Feliz 

Nas suas próximas férias você decidiu passar uma semana com seus cinco melhor amigos da faculdade. Voces alugaram uma casa próximo à praia e também com vários outros atrativos próximos. Voces se reuniram antes e resolveram que voces iriam ficar felizes durante todo o tempo que estivessem na casa, logo, cada pessoa que desejasse entrar na casa tinha que provar que estava feliz naquele momento.

<img src="./images/casaFeliz.jpeg" style="width:350px;height:270px;">
<caption><center> <u> <font color='purple'> **Figura 1** </u><font color='purple'>  : **a casa Feliz**</center></caption>


Como você é um especialista em aprendizado profundo, para ter certeza que as pessoas que entrassem na casa estavam felizes, você resolveu construir um algoritmo que tira uma foto da pessoa na porta da casa e verifica se a pessoa está feliz ou não. A porta será aberta automaticamente caso a pessoa esteja feliz.   

Você pegou fotos de seus amigos e suas obtidas da câmera da porta da frente da casa. A base de dados está rotulada.  

<img src="./images/house-members.png" style="width:550px;height:250px;">

Execute o código abaixo para normalizar a base de dados e conhecer os formatos utilizados. 

In [ ]:
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()

# Normaliza os vetores de imagens
X_train = X_train_orig/255.
X_test = X_test_orig/255.

# Ajusta o formato
Y_train = Y_train_orig.T
Y_test = Y_test_orig.T

print ("número de exemplos de treinamento = " + str(X_train.shape[0]))
print ("número de exemplos de teste = " + str(X_test.shape[0]))
print ("Formato do X_train: " + str(X_train.shape))
print ("Formato do Y_train: " + str(Y_train.shape))
print ("Formato do X_test: " + str(X_test.shape))
print ("Formato do Y_test: " + str(Y_test.shape))

**Detalhes da base de dados "Feliz"**:
- Imagens estão no formato (64,64,3)
- Treinamento: 600 fotos
- Teste: 150 fotos

Agora é a  hora de resolver o problema de entrada na casa Feliz.

## 2 - Construindo um modelo em Keras

Keras é muito bom para criar um protótipo rapidamente. Em um curto espaço de tempo você será capaz de construir um modelo com ótimos resultados.

Aqui está um exemplo de modelo em Keras:

```python
def model(formato_entrada):
    # Define o placeholder de entrada como um tensor com formato formato_entrada. Pense nisto como a imagem de entrada!
    X_input = Input(formato_entrada)

    # Zero-Padding: coloca zeros nas bordas de X_input
    X = ZeroPadding2D((3, 3))(X_input)

    # Aplica um bloco CONV -> BN -> RELU em X
    X = Conv2D(32, (7, 7), strides = (1, 1), name = 'conv0')(X)
    X = BatchNormalization(axis = 3, name = 'bn0')(X)
    X = Activation('relu')(X)

    # Faz o MAXPOOL
    X = MaxPooling2D((2, 2), name='max_pool')(X)

    # Vetoriza X (converte X em um vetor) + FULLYCONNECTED
    X = Flatten()(X)
    X = Dense(1, activation='sigmoid', name='fc')(X)

    # Cria um modelo. Esta linha cria uma instância de modelo em Keras. Utilize esta instância para treinar/testar seu modelo. 
    model = Model(inputs = X_input, outputs = X, name='ModeloFeliz')
    
    return model
```

Note que Keras utiliza uma conversão diferente com nomes de variáveis do que vinhamos utilizando em TensorFlow e Numpy. Em particular, ao invés de criar e atribuir uma nova variável em cada etapa da propagação para frente, como `X`, `Z1`, `A1`, `Z2`, `A2`, etc. para a computação de camadas diferentes, em Keras cada linha de código acima reatribui a `X` um novo valor utilizando `X = ...`. Em outras palavras, durante cada etapa da propagação para frente, estamos utilizando a mesma variável durante toda a computação. A única exceção foi `X_input`, na qual mantivemos separado e não sobrescrevemos, pois precisamos dela para criar o modelo em Keras no final (`model = Model(inputs = X_input, ...)`). 

**Exercício**: Implemente o modelo `ModeloFeliz()`. Esta tarefa, diferente das demais, não possui uma solução definida. Sugerimos que você tente implementar um modelo utilizando a arquitetura indicada e execute até o final desta tarefa usando o modelo inicial,porém, após isto, retorne a este ponto e tente outras arquiteturas. Por exemplo, modifique a arquitetura da rede e tente outros hiper-parâmetros, então modifique a arquitetura e hiper-parâmetros e veja o que você obtem de resposta. Você pode ainda utilizar outras funções como `AveragePooling2D()`, `GlobalMaxPooling2D()`, `Dropout()`. 

**Nota**: Você deve tomar cuidado com o formato dos dados. Utilize o que foi dito em aula para ter certeza que as camadas convolucional, pooling e fully-connected estão corretas com os volumes que você está aplicando.

In [ ]:
# FUNÇÃO DE AVALIAÇÃO: modeloFeliz

def modeloFeliz(input_shape):
    """
    Implementa o modeloFeliz.
    
    Argumento:
    input_shape -- o formato das imagens da base de dados

    Returna
    model -- uma instância do Keras que é um modelo
    """
    
        ### INICIE O SEU CÓDIGO AQUI ###
    # Sinta-se a vontade para utilizar a sequência dada acima para criar o seu modelo, e utilizá-lo em todo o exercício
    # (incluindo o restante deste notebook). Volte a este ponto se você quiser testar outras arquiteturas. 
    # Define o placeholder de entrada como um tensor com formato input_shape. Este é o tamanho da imagem de entrada!
    X_input = Input(formato_entrada)

    # Zero-Padding: cria um padding com zeros ao redor da entrada
    X = ZeroPadding2D((3, 3))(X_input)

    # Sequência de blocos aplicados a X: CONV -> BN -> RELU 
    X = Conv2D(32, (7, 7), strides = (1, 1), name = 'conv0')(X)
    X = BatchNormalization(axis = 3, name = 'bn0')(X)
    X = Activation('relu')(X) 

    # Faz o MAXPOOL
    X = MaxPooling2D((2, 2), name='max_pool')(X)

    # Vetoriza X (converte X em um vetor) + FULLYCONNECTED
    X = Flatten()(X)
    X = Dense(1, activation='sigmoid', name='fc')(X)

    # Cria o modelo. Esta linha cria uma instância do modelo em Keras. Use esta instância para treinar/testar o modelo.
    model = Model(inputs = X_input, outputs = X, name='ModeloFeliz')
    
    ### TÉRMINO DO CÓDIGO ###
    
    return model

Você acabou de construir uma função que descreve seu modelo. Para treinar e testar este modelo são necessárias quatro etapas em Keras:
1. Criar omodelo chamando a função acima.
2. Compilar o modelo com a chamada: `model.compile(optimizer = "...", loss = "...", metrics = ["accuracy"])`
3. Trainar o modelo nos dados de treinamento fazendo a chamada: `model.fit(x = ..., y = ..., epochs = ..., batch_size = ...)`
4. Testar o modelo nos dados de teste fazendo a chamada: `model.evaluate(x = ..., y = ...)`

Se você quiser saber mais sobre `model.compile()`, `model.fit()`, `model.evaluate()` e seus argumentos, verifique a documentação oficial [documentação Keras](https://keras.io/models/model/).

**Exercício**: Implemente a etapa 1, isto é, crie o modelo.

In [ ]:
### INICIE O CÓDIGO AQUI ### (1 line)
ModeloFeliz = 
### TÉRMINO DO CÓDIGO ###

**Exercício**: Implemente a etapa, compile o modelo para configurar o processo de aprendizado. Escolha os 3 argumentos de `compile()` adequadamente. Dica: o desafio feliz é um problema de classificação binária.

In [ ]:
### INICIE O CÓDIGO AQUI ### (1 line)

### TÉRMINO DO CÓDIGO ###

**Exercício**: Implemente a etapa 3, treine o seu modelo. Escolha o número de épocas e o tamanho dos batchs. 

In [ ]:
### INICIE O CÓDIGO AQUI ### (1 line)

### TÉRMINO DO CÓDIGO ###

Note que se você executar `fit()` novamente, o `model` irá continuar a treinar com os parâmetros que ele já havia aprendido ao invés de reinicializá-los.

**Exercício**: Implemente a etapa 4 e test/avalie o seu modelo.

In [ ]:
### INICIE O CÓDIGO AQUI ### (1 line)
preds = 
### TÉRMINO DO CÓDIGO ###
print()
print ("Perda = " + str(preds[0]))
print ("Precisão no teste = " + str(preds[1]))

Se a sua função `modeloFeliz()` funcionou, você deve ter observado uma precisão maior do que uma escolha aleatória (50%) tanto no conjunto de treinamento como no de teste. 

Para que você tenha um ponto de comparação, o modelo deve dar algo em torno de **95% de precisão em 20 épocas no teste** (e da ordem de 99% no treinamento) utilizando um tamanho de mini batch de 16 e o otimizador "adam". Note porém que o modelo já obtem uma precisão razoável com coisa da ordem de 2-5 épocas. Se você for comparar modelos diferentes você pode treiná-los em apenas algumas épocas e verificar o seus desempenhos. 

Sepor acaso você não conseguiu uma boa precisão (maior que 80%), aqui vão algumas dicas do que você pode fazer para melhorar seu modelo:

- Tente utilizar blocos: CONV->BATCHNORM->RELU:
```python
X = Conv2D(32, (3, 3), strides = (1, 1), name = 'conv0')(X)
X = BatchNormalization(axis = 3, name = 'bn0')(X)
X = Activation('relu')(X)
```
até que as dimensões de altura e largura sejam pequenas e o número de canais seja grande (por exemplo ≈32). Você está codificando informação útil em um volume com muitos canais. Voc6e pode então vetorizar este volume  e utilizar uma camada totalmente conectada. 
- Você pode utilizar MAXPOOL após estes blocos. Isto ajudará a reduzir as dimensões em altura e largura.
- Mude o otimizador. Para o meu modelo Adam funcionou bem. 
- Se o modelo estiver com probelmas de execução e voc6e estiver com problemas de memória, reduza o tamanho do batch (12 é um número adequado). 
- Execute por mais épocas, até que a precisão alcance um plato.  

Mesmo que você tenha obtido uma precisão boa, sinta-se a vontade para brincar com o modelo e tentar obter melhores resultados.  

**Nota**: Se você fizer ajustes de hiper-parâmetros em seu modelo, o conjunto de teste se torna um conjunto de desenvolvimento e seu modelo pode superajustar para o conjunto de teste (desenvolvimento). Nesta tarefa não estaremso nos preocupando muito com isso. 


## 3 - Conclusões

Parabéns, você acabou de resolver o desafio da Casa Feliz!!! 

Agora é só utilizar este modelo junto com  a câmera de entrada da casa. Nós não nos preocuparemso aqui com estes detalhes :-).

<font color='blue'>
**O que você deve lembrar desta tarefa:**
- Keras é uma ferramenta recomendada para prototipação rápida. Ela permite testar rapidamente diversos modelos com arquiteturas diferentes. Existe alguma aplicação na sua vida real que você gostaria de tentar utilizar Keras?  
- Lembre-se de como codificar um modelo em Keras e as quatro etapas que levam a avaliação do modelo no conjunto de teste: Criar->Compilar->Ajustar/Treinar->Avaliar/Testar.

## 4 - Outras funções úteis em Keras.

Duas coisas básicas em Keras e que você pode achar útil são: 
- `model.summary()`: imprime os detalhes das camadas do modelo em uma tabela com o tamanho de cada entrada e saída. 
- `plot_model()`: plota o grafo de computação em um layout que você pode ver e salvar em formato ".png" usando SVG() caso você queira compartilhar o modelo. 

Execute o código abaixo para ver o resumo do seu modelo e pesquise na internet para ver como plotar o seu modelo.

In [ ]:
ModeloFeliz.summary()